# Store data

As calling API everytime may takes a while and we may hit the limit of API, we will store the data from API for now. 

In [9]:
import tmdbsimple as tmdb
tmdb.API_KEY = "71e259894a515060876bab2a33d6bdc9"

In [10]:
from imdb import IMDb
import pandas as pd
from PIL import Image
from StringIO import StringIO
import requests
import os
import time
from shutil import copyfile

In [11]:
dir_python_notebook = os.getcwd()
dir_movie_project = os.path.abspath(os.path.join(dir_python_notebook, os.pardir))
dir_data = os.path.join(dir_movie_project, 'data')

In [ ]:
tmdb_filename = str(dir_data)+'\\drv_tmdb_movie_details.json'
tmdb_movies = pd.read_json(tmdb_filename)
tmdb_movies

# IMDB

In [22]:
imdb = IMDb()
#get a movie by id
imdb_movie = imdb.get_movie('0325980')
#access attributes of the movie by dictionary keys
imdb.update(imdb_movie)
#available keys
imdb_info_column  =imdb_movie.keys()
imdb_info_column  = [str(c) for c in imdb_info_column]
imdb_info_column.sort()

In [ ]:
# IMDB API shows that we can retrieve information from proprty of movies object
imdb_info_column = imdb_movie.keys_alias.keys() 
imdb_info_column.sort()
print(len(imdb_info_column))
print(imdb_info_column)

In [ ]:
# IMDB API shows that we can retrieve information from proprty of movies object
imdb_info_column = imdb_movie.keys_alias.keys() 
imdb_info_column.sort()
print(len(imdb_info_column))
print(imdb_info_column)

In [ ]:
# get IMDB movies that we already have data for TMDB
imdb_ids = tmdb_movies['imdb_id']
imdb_ids = [ str(imdb_id.replace('tt','')) for imdb_id in imdb_ids]
imdb_ids[0]

In [ ]:
imdb_info_column = imdb_movie.keys_alias.keys() 
if 'imdb_id' not in imdb_info_column:
    imdb_info_column.append('imdb_id')
imdb_info_column.sort()
movies = pd.DataFrame(columns=imdb_info_column)
imdb_filename = str(dir_data)+'\\drv_imdb_movie_details.json'
imdb_filename_backup = str(dir_data)+'\\drv_imdb_movie_details_bkp.json'

count = 0
# if we already have a movie data file, we can just continue appending it instead of starting from id 1 again.
imdb_id_set = set(imdb_ids)
if os.path.isfile(imdb_filename):
    movies = pd.read_json(imdb_filename)
    if len(movies['imdb_id']) >0:
        imdb_id_set = imdb_id_set.difference(set(movies['imdb_id']))
        imdb_ids = list(imdb_id_set)
        imdb_info_column = movies.columns
    else:
        movies = pd.DataFrame(columns=imdb_info_column)

for i in imdb_ids[0:]:
    count += 1

    #if (count % 40 == 0):
    #    time.sleep(10) # make sure we do not hit API limit
    if (count % 1000 == 0):
        movies.to_json(path_or_buf= imdb_filename)
    #skip the non-existing movie ids
    #print(i)
    try:
        imdb_movie = imdb.get_movie(i)
        imdb.update(imdb_movie)
        
        movie_details = []
        for c in imdb_info_column:
            if imdb_movie.has_key(c) :
                if type(imdb_movie[c]) is list:                    
                    if isinstance(imdb_movie[c][0], ib.Person.Person):
                        info_list = []
                        for item in imdb_movie[c]:
                            info = {}
                            info["personID"] = item.getID()
                            info["name"] = item.get("name")
                            info_list.append(info)
                        movie_details.append(info_list)
                    elif isinstance(imdb_movie[c][0], ib.Company.Company):
                        info_list = []
                        for item in imdb_movie[c]:
                            info = {}
                            info["companyID"] = item.getID()
                            info["name"] = item.get("name")
                            info_list.append(info)
                        movie_details.append(info_list)  
                    else:
                        movie_details.append(imdb_movie[c])
                else:
                    movie_details.append(imdb_movie[c])
            else:
                if c == "imdb_id":
                    movie_details.append(i)
                else:
                    movie_details.append(None)
                
        movies.loc[len(movies.index)] = movie_details
        
    except Exception:
        print(i)
        continue
        
movies.to_json(path_or_buf= imdb_filename)


In [ ]:
#make a backup in case of corruption
copyfile(imdb_filename, imdb_filename_backup) 

In [ ]:
imdb_movies = pd.read_json(imdb_filename)
imdb_movies